ASAP CRN Metadata compilation

# Team Hardy. ASAP CRN Metadata scrubbing

confirm Team Hardy scRNAseq metadata
9 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table, create_metadata_package
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, intervention_typer
from checksums import extract_md5_from_details2, get_md5_hashes 
from bucket_util import authenticate_with_service_account, gsutil_ls, gsutil_cp, gsutil_mv 

%load_ext autoreload
%autoreload 2

root_path = Path.home() / ("Projects/ASAP/data/teams")


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

```python
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")
```


## Load original tables 
These were submitted as v1.0

Team Hardy

In [3]:
## convert 
team = "hardy"
dataset_name = "sn-rnaseq"


metadata_path = root_path / f"{team}/{dataset_name}/metadata"
og_path = metadata_path / "og"


# v1_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'CLINPATH', 'SAMPLE']

# dfs = {}
# for table in v1_tables:
#     dfs[table] =read_meta_table(og_path / f"{table}.csv")


STUDY = read_meta_table(og_path / "STUDY.csv")
PROTOCOL = read_meta_table(og_path / "PROTOCOL.csv")
SUBJECT = read_meta_table(og_path / "SUBJECT.csv")
CLINPATH = read_meta_table(og_path / "CLINPATH.csv")
SAMPLE = read_meta_table(og_path / "SAMPLE.csv")


metadata_version = "v1"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [4]:
STUDY

,name,value
0,project_name,Understanding mechanisms of Parkinson's diseas...
1,project_dataset,Hardy snRNA-seq
2,project_description,Genetic analysis has identified many risk gene...
3,ASAP_team_name,TEAM-HARDY
4,ASAP_lab_name,Ryten Lab
5,PI_full_name,Mina Ryten
6,PI_email,mina.ryten@ucl.ac.uk
7,contributor_names,"Aine Fairbrother-Browne, Jonathan Brenton, Mel..."
8,submitter_name,Aine Fairbrother-Browne
9,submitter_email,aine.fairbrother-browne.18@ucl.ac.uk


In [5]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 1, 09/2023"


In [6]:

tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
STUDY.head()


,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 1, 09/2023"


In [7]:
STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.reset_index(drop=True, inplace=True)
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,v1_20241101


In [8]:
PROTOCOL

,name,value
0,sample_collection_summary,"This dataset contains cortical regions only, p..."
1,cell_extraction_summary,From protocols.io: This protocol is used to is...
2,lib_prep_summary,'Nuclei were extracted from homogenised post-m...
3,data_processing_summary,Cell ranger was used to convert raw sequencing...
4,github_url,Raw to fastq to mapped: https://github.com/RHR...
5,protocols_io_DOI,Nuclear extraction protocol: 10.17504/protocol...
6,other_reference,NA


In [9]:
# fix PROTOCOL formatting
tmp = pd.DataFrame()
tmp = PROTOCOL[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
PROTOCOL = tmp.drop([0])
PROTOCOL.reset_index(drop=True, inplace=True)
PROTOCOL.head()


,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,"This dataset contains cortical regions only, p...",From protocols.io: This protocol is used to is...,'Nuclei were extracted from homogenised post-m...,Cell ranger was used to convert raw sequencing...,Raw to fastq to mapped: https://github.com/RHR...,Nuclear extraction protocol: 10.17504/protocol...,NA


In [10]:
SAMPLE[['assay','sequencing_end', 'sequencing_length', 'sequencing_instrument']]

,assay,sequencing_end,sequencing_length,sequencing_instrument
0,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
1,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
2,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
4,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
...,...,...,...,...
3611,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3612,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3613,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3614,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000


In [11]:

# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

SAMPLE


# add 'replicate' coding (nans)

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
1,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
2,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
3,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
4,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA
3612,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA
3613,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA
3614,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA


In [12]:
SAMPLE['replicate'] = SAMPLE['replicate'].replace({NULL: "Rep1", "rep1": "Rep1", "rep2": "Rep2"})

In [ ]:
# 🚨⚠️❗ the source_subject_id for this bulk set overlap with the sn Hardy dataset.  However
# that dataset specifies different internal subject_ids (e.g. babom, borah, etc.)  Likewise, the sample_id are 
# inconsistent between the two datasets.  Fortunately, the source_subject_id saves us.  ASAP ID generation
# works correctly, and sample / subject IDs are correct at the ASAP_ID level.

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
1,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
2,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
3,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
4,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA


In [14]:
CLINPATH['path_braak_asyn']

0       6
1       6
2       6
3       6
4       0
       ..
133    NA
134    NA
135    NA
136    NA
137    NA
Name: path_braak_asyn, Length: 138, dtype: object

In [15]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_nft'] = CLINPATH['path_braak_nft'].replace({"0":"0", 
                                                                "1":"I", 
                                                                "2": "II", 
                                                                "3":"III", 
                                                                "4":"IV", 
                                                                "5":"V", 
                                                                "6":"VI"})


In [16]:

# code family_history as "Not Reported" (currently empty)
CLINPATH['family_history'] = "Not Reported"



# check APOE_e4_status ? currently empty

# `path_autopsy_dx_main`  actually seems good parser might be wrong

# code "at least 4" as "4/5" 

CLINPATH['path_thal'] = CLINPATH['path_thal'].replace({'At least 4':"4/5"})


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace({'Diffuse neocortical': "Diffuse, neocortical (brainstem, limbic and neocortical involvement)", 
                                                        'Limbic transitional': "Limbic (transitional)" ,
                                                        'Diffuse Neocortical':"Diffuse, neocortical (brainstem, limbic and neocortical involvement)"})


In [17]:

CLINPATH['path_nia_aa_a'] = CLINPATH['path_nia_aa_a'].replace({"0":"A0", 
                                                                    "1":"A1", 
                                                                    "2": "A2", 
                                                                    "3":"A3"})


In [18]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_b'] = CLINPATH['path_nia_aa_b'].replace({"0":"B0", 
                                                                "1":"B1", 
                                                                "2": "B2", 
                                                                "3":"B3"})


# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_c'] = CLINPATH['path_nia_aa_c'].replace({"0":"C0", 
                                                                "1":"C1", 
                                                                "2": "C2", 
                                                                "3":"C3"})



In [19]:



CLINPATH['path_ad_level'] = CLINPATH['path_ad_level'].replace({"No evidence": "No evidence of Alzheimer\'s disease neuropathological change"})





In [20]:
path_autopsy_map = { "Parkinson's disease with dementia": "Parkinson's disease with dementia", 
       "Parkinson's disease": "Parkinson's disease",
       'Control brain':"Control, no misfolded protein or significant vascular pathology", 
       'Pathological ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Control brain / Path ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Argyrophilic grain disease': "Control, Argyrophilic grain disease",
       'Control brain, Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       'Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       "Control brain, Alzheimer`s disease (intermediate level AD pathological change)":"Alzheimer's disease (intermediate level neuropathological change)",
       'Control brain / Path ageing, CAA':"Control, Cerebrovascular disease (cerebral amyloid angiopathy)"}


In [21]:
CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_map)
# - _*age_at_diagnosis*_:  invalid values 💩'69', '61', 'Not applicable', '71', '60', '57', '79', '87', '67', '75', '50', '35', '44', '42', '58', '80', '72', '65', '48', '74', '62', '36', '77', '43', '56', '55', '37', '66', '64', '49', 'NA'

CLINPATH['age_at_diagnosis'] = CLINPATH['age_at_diagnosis'].replace({"Not applicable": NULL})

In [22]:
# 🚨⚠️❗ **Extra field in CLINPATH: source_subject_id**
# rename source_subject_id to source_sample_id
CLINPATH.rename(columns={"source_subject_id":"source_sample_id"}, inplace=True)


In [23]:
SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({"Alzheimer?s disease":"Alzheimer's disease"})

In [24]:
# - _*molecular_source*_:  invalid values 💩'nuclear RNA'
#     - valid ➡️ 'Total RNA', 'PolyA RNA', 'Cytoplasmic RNA', 'Nuclear RNA', 'Genomic DNA', 'Protein', 'Other'
# - _*input_cell_count*_:  invalid values 💩'16483.800000000003', '20418', '16714.643750000003', '17644.912977099237', '17353.600000000002', '17388.800000000003', '17952', '17776', '19712', '16948.800000000003', '17098', '18656', '17265.600000000002', '12223.199999999999', '14520', '18128', '17933.2', '12289.199999999999', '12051.599999999999', '13860', '13970', '18304', '17564.879999999997', '18480', '17916.800000000003', '19855.07096774194', '17368.707692307697', '16766', '17529.600000000002', '19360', '23100', '15180', '18100.800000000003', '19536', '18832', '13094.4', '18601.559999999998', '13194.72', '13226.4', '11893.199999999999', '16632', '20400', '21800', '17555.9', '15312', '12249.599999999999', '18426', '23738.000000000004', '17920', '18347.28', '18503.76', '16859', '19680', '18163.2', '20856', '15048', '14124', '17600', '12381.599999999999', '16566.800000000003', '19422', '21120', '17300.800000000003', '21472.100000000002', '45500.600000000006', '17001.600000000002', '17314', '10942.8', '9292.8', '20064', '18008.199999999997', '17476.800000000003', '12157.199999999999', '17688', '13570.656', '13572.24', '18249.48', '16684.800000000003', '13678.5', '17716.83', '18458', '25600', '23800', '13195.775999999998', '13190.099999999999', '18240', '17762', '19920', '18445.079999999998', '17738.6', '16772.800000000003', '17477.7', '19520', '19754', '17211.828571428574', '14388', '15582', '22400', '22000', '18260', '14552.999999999998', '13332', '19928.300000000003', '19256', '19008', '18560', '17760', '20617.2', '18053.879999999997', '27365.100000000002', '17885.2'
#     - valid ➡️ int or NULL ('NA')
# - _*sequencing_end*_:  invalid values 💩'paired-end'
#     - valid ➡️ 'Single', 'Paired-end'
# - _*sequencing_length*_:  invalid values 💩'190'
#     - valid ➡️ '25', '50', '100', '150'
# - _*technology*_:  invalid values 💩'sN'
#     - valid ➡️ 'SN', 'SC', 'Spatial', 'Bulk', 'Seq', 'Array'
# - _*adjustment*_:  invalid values 💩'raw'
#     - valid ➡️ 'Raw', 'Processed'
# - _*content*_:  invalid values 💩'reads'
#     - valid ➡️ 'Counts', 'Probabilities', 'Genotypes', 'Dosages', 'Reads'

SAMPLE = prep_table(SAMPLE, CDEv1)

In [25]:
v1_path = metadata_path / "v1"
STUDY.to_csv(v1_path / f"STUDY.csv", index=False)
SAMPLE.to_csv(v1_path / f"SAMPLE.csv", index=False)
SUBJECT.to_csv(v1_path / f"SUBJECT.csv", index=False)
PROTOCOL.to_csv(v1_path / f"PROTOCOL.csv", index=False)
CLINPATH.to_csv(v1_path / f"CLINPATH.csv", index=False)

In [26]:
SAMPLE[['sample_id','subject_id','file_name','file_MD5']]['file_MD5'][0]

'f37738d04879c1cb8c380920964dad4d'

## validate v1


In [27]:
CDE = CDEv1
tables = CDE['Table'].unique()
v1_path = metadata_path / "v1"

dfs = {}
for table in tables:
    df = read_meta_table(v1_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table
    # df.to_csv(v1_path / f"{table}.csv", index=False)

recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **4 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- race: 64/64 empty rows (REQUIRED)

	- ethnicity: 64/64 empty rows (REQUIRED)

	- duration_pmi: 1/64 empty rows (REQUIRED)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
🚨⚠️❗ **Missing Required Fields in SAMPLE: source_sample_id**
🚨⚠️❗ **Missing Optional Fields in SAMPLE: DV200**
🚨⚠️❗ **9 Fields with empty (NULL) values:**

	- source_RIN: 3616/3616 empty rows (REQUIRED)

	- header: 3616/3616 empty rows (REQUIRED)

	- annotation: 3616/3616 

In [28]:
schema = CDE[CDE['Table'] == "STUDY"]
schema

,Table,Field,Description,DataType,Required,Validation,Shared_key
0,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN,0
1,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN,0
2,STUDY,project_description,Project Description: Brief description of th...,String,Required,NaN,0
3,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-...",0
4,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN,0
5,STUDY,PI_full_name,Principal Investigator Name.: Full name of t...,String,Required,NaN,0
6,STUDY,PI_email,Principal Investigator E-mail.: E-mail addre...,String,Required,NaN,0
7,STUDY,contributor_names,Contributor(s).: Names of all contributors t...,String,Required,NaN,0
8,STUDY,submitter_name,Data Submitter Name.: Full name of individua...,String,Required,NaN,0
9,STUDY,submitter_email,Data Submitter E-Mail.: Email address of ind...,String,Required,NaN,0


In [29]:
STUDY

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,v1_20241101


In [30]:
# make tables conform to CDE and save extra columns as "auxiliary"

for table in tables:
    df = dfs[table]
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v1_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v1_path / f"{table}.csv", index=False)
    

Saved SAMPLE_auxiliary.csv


In [31]:
STUDY

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,v1_20241101


## v1->v2

In [32]:
v2_path = metadata_path / "v2"

# rationalize the dataset id
team_dataset_id = dataset_name.replace(" ", "_").replace("-", "_")

v2_tables, aux_tables = v1_to_v2(v1_path, v2_path, CDEv1, CDEv2, team_dataset_id=team_dataset_id)



recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int


## validate v2 tables


In [33]:
CDE = CDEv2
for table,df in v2_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **5 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ **15 Fields with empty (NULL) values:**

	- AMPPD_id: 74/74 empty rows (REQUIRED)

	- GP2_id: 2/74 empty rows (REQUIRED)

	- race: 74/74 empty rows (REQUIRED)

	- ethnicity: 74/74 empty rows (REQUIRED)

	- last_diagnosis: 74/74 empty rows (R

## v2->v3

In [34]:
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

f"{v3_meta_tables}"

"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']"

In [35]:
v3_path = metadata_path / "v3"

v3_tables, aux_tables = v2_to_v3_PMDBS(v2_path, v3_path, CDEv2, CDEv3)

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
recoding replicate_count as int
recoding repeated_sample as int


In [36]:
v3_tables["STUDY"]


,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,sn_rnaseq,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,...,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,NA,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,Https://github.com/RHReynolds/asap-pd-progress...,v3.0_20241101,NA


### validate v3 tables


In [37]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **5 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- race: 64/64 empty rows (REQUIRED)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **12 Fields with empty (NULL) values:**

	- source_sample_id: 138/138 empty rows (REQUIRED)

	- organism: 138

In [38]:
STUDY = v3_tables['STUDY']
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,sn_rnaseq,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,...,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,NA,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,Https://github.com/RHReynolds/asap-pd-progress...,v3.0_20241101,NA


-------------------------
## check md5s



In [40]:
print(team)

source = "pmdbs"

bucket = f"asap-raw-team-{team}-{source}-{dataset_name}"


key_file_path = Path.home() / f"Projects/ASAP/{team}-credentials.json"

res = authenticate_with_service_account(key_file_path)
print(res)
prefix = "fastqs/**/*.gz"
bucket_files_md5 = get_md5_hashes( bucket, prefix)

hardy
CompletedProcess(args='gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json', returncode=0, stdout='', stderr='Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]\n')
gsutil -u dnastack-asap-parkinsons hash -h gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/fastqs/**/*.gz


In [41]:

checksum = v3_tables['DATA'][['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5']
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)
checksum[checksum.check1 != checksum.check2].file_name.to_list()


/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16379/2388309125.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5']
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_16379/2388309125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


[]

_____

## prep metadata in raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/upload
- 2. copy metadata/upload to dataset upload (upload subdir)

In [44]:
metadata_subdir = "metadata"
current_files = gsutil_ls(bucket,metadata_subdir)

gsutil -u dnastack-asap-parkinsons ls gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata
gsutil command succeeded: gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/CLINPATH.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/PROTOCOL.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/SAMPLE.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/STUDY.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/SUBJECT.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/v2-processed/
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/v2/



In [45]:
metadata_subdir2 = "metadata/upload"
bucket = current_files[0].split("/")[2]

for file in current_files:
    if file == "":
        continue
    file_nm = Path(file).name
    is_dir = not file_nm.__contains__(".")

    source = f"gs://{bucket}/{metadata_subdir}/{file_nm}"

    destination = f"gs://{bucket}/{metadata_subdir2}/{file_nm}"
    gsutil_mv(source, destination, is_dir)

gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/CLINPATH.csv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/CLINPATH.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/PROTOCOL.csv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/PROTOCOL.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/SAMPLE.csv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/SAMPLE.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/STUDY.csv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/STUDY.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/SUBJECT.csv gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/SUBJECT.csv
gsutil command succeeded: 
gsutil -u dnastack-asa

In [46]:

metadata_subdir = "metadata/upload"
current_files = gsutil_ls(bucket,metadata_subdir)

gsutil -u dnastack-asap-parkinsons ls gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload
gsutil command succeeded: gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/CLINPATH.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/PROTOCOL.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/SAMPLE.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/STUDY.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/SUBJECT.csv
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/v2-processed/
gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload/v2/



Archive the uploaded metadata locally



In [49]:
file_source = f"gs://{bucket}/{metadata_subdir}"
destination = f"{metadata_path}"

In [50]:

is_dir = True
gsutil_cp(file_source, destination, is_dir)

gsutil -u dnastack-asap-parkinsons cp -r gs://asap-raw-team-hardy-pmdbs-sn-rnaseq/metadata/upload /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata
gsutil command succeeded: 


''

--------------------
## Create metadata package

This will copy the final updated to v3.0 metadata to `asap-could-processing-resources`


In [51]:
metadata_source = metadata_path


source = "pmdbs"
archive_root = Path.home() / "Projects/ASAP/asap-crn-metadata/datasets"
dataset_path = archive_root / f"{team}-{source}-{dataset_name}"
# bucket = f"asap-raw-data-team-{team}" # for now old locations
metadata_source, dataset_path

(PosixPath('/Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata'),
 PosixPath('/Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq'))

________

In [52]:
fnms = create_metadata_package(metadata_source, dataset_path)


Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/v1 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/v1
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/v2 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/v2
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/v3 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/v3
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/og to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/og
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/upload to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/upload


_____

generate ASAP IDs + transfering back to raw data bucket via `asap-crn-metadata` 

_____

## transfer metadata to raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/archive
- 2. copy package to metadata/ . i.e. /og/*.csv, /v??/*.csv

______________

In [ ]:
# these are actually matches... probably unstripped line endings.